In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams
sc.set_figure_params(dpi=300, color_map='viridis')
sc.settings.verbosity = 2

In [ ]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
from glob import iglob
from tqdm import tqdm

In [ ]:
%matplotlib inline

In [ ]:
##scanpy2matrix
adata= sc.read("Fig4a.h5")
def anndata_to_10x(adata:sc.AnnData, _dir):
    from scipy.io import mmwrite
    from os.path import join, exists
    from os import makedirs
    if not exists(_dir):
        makedirs(_dir)
    X = adata.X.T.tocoo()
    mmwrite(join(_dir, 'matrix.mtx'), X)
    gene_df = adata.var
    gene_df['ENS-0-0'] = gene_df.index
    gene_df.index = gene_df.index.values.tolist() 
    gene_df.drop('ENS-0-0', axis=1, inplace=True)
    gene_df.to_csv(join(_dir, 'genes.tsv'), sep='\t', header=False, index=True)
    adata.obs.to_csv(join(_dir, 'barcodes.tsv'), sep='\t', header=False, index=True)

anndata_to_10x(adata, './scanpy_10x')

In [ ]:
###write metadata
cell_metadata = adata.obs
cell_metadata
cell_metadata.to_csv("cell_metadata.csv", index=True)

In [ ]:
%%R
Sys.setenv(LANGUAGE = "en")
options(stringsAsFactors = FALSE)
library(Seurat)
data <- Read10X(data.dir ="~/scanpy_10x")
seurat_obj <- CreateSeuratObject(counts = data, min.cells = 0, min.features = 0, assay = 'RNA')
metadata <- read.csv("cell_metadata.csv",header=T,row.names=1)
seurat_obj@meta.data<-metadata
saveRDS(seurat_obj,file="seurat_obj.rds")
##split by region
seurat_obj <- readRDS("seurat_obj.rds")
sce <- SplitObject(seurat_obj, split.by = "region")
sce
Idents(sce[["Prefrontal cortex"]])<-"Celltype"
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
Idents(sce[["Prefrontal cortex"]]) <- factor(Idents(sce[["Prefrontal cortex"]]), levels= My_levels)
levels(Idents(sce[["Prefrontal cortex"]]))
saveRDS(sce[["Prefrontal cortex"]],file="PFC_input.rds")

##region2
Idents(sce[["Hippocampus"]])<-"Celltype"
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
Idents(sce[["Hippocampus"]]) <- factor(Idents(sce[["Hippocampus"]]), levels= My_levels)
levels(Idents(sce[["Hippocampus"]]))
saveRDS(sce[["Hippocampus"]],file="Hippo_input.rds")

rm(list=ls())
gc()
library(CellChat)
library(ggplot2)                  
library(patchwork)
library(igraph)
PFC <- readRDS("PFC_input.rds")
PFC.data.input <- GetAssayData(PFC, assay = "RNA", slot = "data")
PFC.identity <- subset(PFC@meta.data, select = "Celltype")
cellchat <- createCellChat(object = PFC.data.input, meta = PFC.identity, group.by = "Celltype")
cellchat <- addMeta(cellchat, meta = PFC.identity, meta.name = "labels")
cellchat <- setIdent(cellchat, ident.use = "labels") 
levels(cellchat@idents)
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
cellchat@idents <- factor(cellchat@idents, levels= My_levels)
levels(cellchat@idents)
groupSize <- as.numeric(table(cellchat@idents))
#Set the ligand-receptor interaction database
CellChatDB <- CellChatDB.human  
showDatabaseCategory(CellChatDB)
#ggsave("cellchat.human.summary.pdf",width = 8, height = 7)
setwd("./Cellchat")
dplyr::glimpse(CellChatDB$interaction)
CellChatDB.use <- subsetDB(CellChatDB, search = c("Secreted Signaling","ECM-Receptor","Cell-Cell Contact" ))
cellchat@DB <- CellChatDB.use 
cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- projectData(cellchat, PPI.human )
cellchat <- computeCommunProb(cellchat)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)
levels(cellchat@idents)
groupSize <- as.numeric(table(cellchat@idents))
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat@net$count, vertex.weight = groupSize, weight.scale = T, label.edge= F, title.name = "Number of interactions",color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
netVisual_circle(cellchat@net$weight, vertex.weight = groupSize, weight.scale = T, label.edge= F, title.name = "Interaction weights/strength",color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))

weight<-as.data.frame(cellchat@net$weight)
write.table(weight,file="weight_Cerebral.xls",sep="\t")
count<-as.data.frame(cellchat@net$count)
write.table(count,file="count_Cerebral.xls",sep="\t")
netP<-as.data.frame(cellchat@netP[["prob"]])
write.table(netP,file="netP_Cerebral.xls",sep="\t")

# Access all the signaling pathways showing significant communications
pathways.show.all <- cellchat@netP$pathways
# check the order of cell identity to set suitable vertex.receiver
levels(cellchat@idents)
vertex.receiver = seq(1,4)
for (i in 1:length(pathways.show.all)) {
  # Visualize communication network associated with both signaling pathway and individual L-R pairs
  netVisual(cellchat, signaling = pathways.show.all[i], vertex.receiver = vertex.receiver, layout = "hierarchy")
  # Compute and visualize the contribution of each ligand-receptor pair to the overall signaling pathway
  gg <- netAnalysis_contribution(cellchat, signaling = pathways.show.all[i])
  ggsave(filename=paste0(pathways.show.all[i], "_L-R_contribution.pdf"), plot=gg, width = 3, height = 2, units = 'in', dpi = 300)
}

##Compute and visualize the network centrality scores
#cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP")
#netAnalysis_signalingRole_network(cellchat, signaling = pathways.show, width = 8, height = 2.5, font.size = 10,color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))

ht1 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing",width = 8, height = 14, color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ht2 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming",width = 8, height = 14,color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ht1 + ht2
#10,15
saveRDS(cellchat,file = 'PFC.cellchat.rds')
##region2
rm(list=ls())
library(CellChat)
library(ggplot2)                  
library(patchwork)
library(igraph)
library(Seurat)
Hippo <- readRDS("Hippo_input.rds")
Hippo.data.input <- GetAssayData(Hippo, assay = "RNA", slot = "data")
Hippo.identity <- subset(Hippo@meta.data, select = "Celltype")
cellchat <- createCellChat(object = Hippo.data.input, meta = Hippo.identity, group.by = "Celltype")
cellchat <- addMeta(cellchat, meta = Hippo.identity, meta.name = "labels")
cellchat <- setIdent(cellchat, ident.use = "labels") 
levels(cellchat@idents)
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
cellchat@idents <- factor(cellchat@idents, levels= My_levels)
levels(cellchat@idents)
groupSize <- as.numeric(table(cellchat@idents))
#Set the ligand-receptor interaction database
CellChatDB <- CellChatDB.human  
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)
CellChatDB.use <- subsetDB(CellChatDB, search = c("Secreted Signaling","ECM-Receptor","Cell-Cell Contact" ))
cellchat@DB <- CellChatDB.use 
cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- projectData(cellchat, PPI.human )
cellchat <- computeCommunProb(cellchat)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)
levels(cellchat@idents)
groupSize <- as.numeric(table(cellchat@idents))
setwd("./Cellchat")
par(mfrow = c(1,2), xpd=TRUE)
netVisual_circle(cellchat@net$count, vertex.weight = groupSize, weight.scale = T, label.edge= F, title.name = "Number of interactions",color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
netVisual_circle(cellchat@net$weight, vertex.weight = groupSize, weight.scale = T, label.edge= F, title.name = "Interaction weights/strength",color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))

weight<-as.data.frame(cellchat@net$weight)
write.table(weight,file="weight_Hippo.xls",sep="\t")
count<-as.data.frame(cellchat@net$count)
write.table(count,file="count_Hippo.xls",sep="\t")
netP<-as.data.frame(cellchat@netP[["prob"]])
write.table(netP,file="netP_Hippo.xls",sep="\t")

# Access all the signaling pathways showing significant communications
pathways.show.all <- cellchat@netP$pathways
# check the order of cell identity to set suitable vertex.receiver
levels(cellchat@idents)
vertex.receiver = seq(1,4)
for (i in 1:length(pathways.show.all)) {
  # Visualize communication network associated with both signaling pathway and individual L-R pairs
  netVisual(cellchat, signaling = pathways.show.all[i], vertex.receiver = vertex.receiver, layout = "hierarchy")
  # Compute and visualize the contribution of each ligand-receptor pair to the overall signaling pathway
  gg <- netAnalysis_contribution(cellchat, signaling = pathways.show.all[i])
  ggsave(filename=paste0(pathways.show.all[i], "_L-R_contribution.pdf"), plot=gg, width = 3, height = 2, units = 'in', dpi = 300)
}

##Compute and visualize the network centrality scores
#cellchat <- netAnalysis_computeCentrality(cellchat, slot.name = "netP")
#netAnalysis_signalingRole_network(cellchat, signaling = pathways.show, width = 8, height = 2.5, font.size = 10,color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ht1 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing",width = 8, height = 14, color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ht2 <- netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming",width = 8, height = 14,color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ht1 + ht2
saveRDS(cellchat,file = 'Hippo.cellchat.rds')
rm(list=ls())
library(CellChat)
library(ggplot2)                  
library(patchwork)
library(igraph)
library(Seurat)
##merge 2 regions
PFC<-readRDS("PFC.cellchat.rds")
PFC <- updateCellChat(PFC)
Hippo<-readRDS("Hippo.cellchat.rds")
Hippo <- updateCellChat(Hippo)
levels(Hippo@idents)
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
Hippo@idents <- factor(Hippo@idents, levels= My_levels)
levels(Hippo@idents)
levels(PFC@idents)
My_levels <- c("Oligodendrocyte precursor", "Oligodendrocyte", "Newly formed oligodendrocytes", "Astrocyte", "Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell", "Fibroblast", "Endothelial", "GABAergic", "Glutamatergic", "Glutamatergic(Pyr)")
PFC@idents <- factor(PFC@idents, levels= My_levels)
levels(PFC@idents)
PFC <- updateCellChat(PFC)
Hippo <- updateCellChat(Hippo)
PFC <- updateCellChat(PFC)
Hippo <- updateCellChat(Hippo)
group.new = levels(Hippo@idents)
PFC <- liftCellChat(PFC, group.new)
object.list <- list(PFC = PFC, Hippo = Hippo)
cellchat <- mergeCellChat(object.list, add.names = names(object.list), cell.prefix = TRUE)
#object.list <- list(PFC = PFC, Hippo = Hippo)
#cellchat <- mergeCellChat(object.list, add.names = names(object.list))
#Fig.7a/b
par(mfrow = c(1,2), xpd=TRUE)
netVisual_diffInteraction(cellchat, weight.scale = T,color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
netVisual_diffInteraction(cellchat, weight.scale = T, measure = "weight",color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
ggsave(filename="interaction_circle2.pdf",width = 12, height = 7)
#Fig.7c/d
weight.max <- getMaxWeight(object.list, attribute = c("idents","count"))
par(mfrow = c(1,2), xpd=TRUE)
for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = T, label.edge= F, color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"),edge.weight.max = weight.max[2], edge.width.max = 12, title.name = paste0("Number of interactions - ", names(object.list)[i]))
}

#Fig.7e
gg1 <- rankNet(cellchat, mode = "comparison", stacked = T, do.stat = TRUE,color.use=c("#B3128A","#74CAFF"))
gg2 <- rankNet(cellchat, mode = "comparison", stacked = F, do.stat = TRUE,color.use=c("#B3128A","#74CAFF"))
p3=gg1 + gg2
p3
ggsave(filename="flow.pdf",plot=p3, width = 12, height = 8)

#Supplemental Table 6.xlsx
g1<-netVisual_bubble(cellchat, sources.use = c(1,2,3,4,5,6,7,8,9,10,11,12), targets.use = c(1,2,3,4,5,6,7,8,9,10,11,12),  comparison = c(1, 2), angle.x = 45,color.text=c("#B3128A","#74CAFF"))
g1data<-as.data.frame(g1$data)
write.table(g1data,file="All_ineration_info.xls",sep="\t")
saveRDS(cellchat,file = 'Merge.cellchat.rds')

#Fig.7f
num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets
gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], title = names(object.list)[i], weight.MinMax = weight.MinMax, color.use=c("#006091", "#5C89CC", "#74caff", "#AFE1AF", "#F0E68C", "#B2E71A", "#8B8000", "#D2B48C", "#50C878", "#E30B5C", "#CC5500", "#FA5F55"))
}

p1=patchwork::wrap_plots(plots = gg)
ggsave(filename="major_scatter.pdf",plot=p1, width = 8, height = 4)

#Fig.7g
gg1 <- netAnalysis_signalingChanges_scatter(cellchat, idents.use = "Microglia(PCDH9high)", color.use = c("#7AB000", "#B3128A","#74CAFF"))
gg1data<-as.data.frame(gg1$data)
write.table(gg1data,file="Microglia(PCDH9high)_scatter_info.xls",sep="\t")
ggsave(filename="Microglia(PCDH9high)_scatter.pdf",plot=gg1, width = 6, height = 4)

#Fig.7h
#from all senders to Microglia2
pairLR.use <- extractEnrichedLR(cellchat, signaling = c("NRG","CADM","NEGR","LAMININ"))
netVisual_bubble(cellchat, sources.use = c(1:12), targets.use = 6, pairLR.use = pairLR.use, remove.isolate = TRUE,  comparison = c(1, 2), angle.x = 45,thresh=0.05)
# filter p-value> 0.05
netVisual_bubble(Hippo, sources.use = c(1:12), targets.use = 6, pairLR.use = pairLR.use, remove.isolate = TRUE)

#Fig.7i
#from Microglia(PCDH9high) to receivers
pairLR.use <- extractEnrichedLR(cellchat, signaling = c("NRG","CADM","NEGR","LAMININ"))
netVisual_bubble(cellchat, sources.use = 6, targets.use = c(1:12), pairLR.use = pairLR.use, remove.isolate = TRUE,  comparison = c(1, 2), angle.x = 45)
# filter p-value> 0.05
netVisual_bubble(Hippo, sources.use = 6, targets.use = c(1:12), pairLR.use = pairLR.use, remove.isolate = TRUE)
